In [46]:
import pandas as pd 
import numpy  as np  
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram    
import sys 
sys.setrecursionlimit(10000)  #increasing maximum recursion from 999 to 10K

from scipy.cluster.hierarchy import dendrogram, fcluster, leaves_list
from scipy.spatial import distance
from fastcluster import linkage
from sklearn.metrics.pairwise import pairwise_distances  
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage , fcluster
from scipy.cluster.hierarchy import cophenet
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation 

import scipy.stats
import math


# importing patients and thier ccs_codes

In [3]:
fields=['new_euid','CLAIM_ID','cli_dx1_ccs','cli_dx2_ccs','cli_dx3_ccs','cli_dx4_ccs','proc_ccs']
type = {'new_euid':str,'CLAIM_ID':str,'cli_dx1_ccs':float,'cli_dx2_ccs':float,'cli_dx3_ccs':float,'cli_dx4_ccs':float,'proc_ccs':str}
all_new=pd.read_csv("../../Data/all_new.csv",sep=",",usecols=fields,dtype=type) #i
all_new.head()   

,new_euid,CLAIM_ID,cli_dx1_ccs,cli_dx2_ccs,cli_dx3_ccs,cli_dx4_ccs,proc_ccs
0,0568848,140830BQR000,138.0,NaN,NaN,NaN,227
1,0568848,141050B0JS00,166.0,176.0,NaN,NaN,227
2,0568848,1430000SKB00,256.0,166.0,259.0,53.0,227
3,0568848,1430000SKB00,256.0,166.0,259.0,53.0,202
4,0568848,143080BZCJ00,51.0,166.0,NaN,NaN,227


In [4]:
print("Total Observations")
print(all_new.shape)                    # Total observatons

print("\n")  

print("Null observation per column")
print(all_new.isnull().sum())           # Null observation per column  

print("\n")

print("unique  patients")
print(len(all_new['new_euid'].unique()))  #Total patients

Total Observations
(3176014, 7)


Null observation per column
new_euid             0
CLAIM_ID             0
cli_dx1_ccs          0
cli_dx2_ccs    2200292
cli_dx3_ccs    2652056
cli_dx4_ccs    2892445
proc_ccs             0
dtype: int64


unique  patients
134054


# subsetting only diabetic patients(i,e CCS either 50.0,49.0) 

In [5]:
#CCS=49.0 Diabetic without complication.  
#CCS=50.0 Diabetic with complication.

all_diabetic=all_new[all_new.cli_dx1_ccs.isin([50.0,49.0])| 
                     all_new.cli_dx2_ccs.isin([50.0,49.0])|
                     all_new.cli_dx3_ccs.isin([50.0,49.0])|
                     all_new.cli_dx4_ccs.isin([50.0,49.0])
                    ]   


print(all_diabetic.shape) 
all_diabetic.head()

(96238, 7)


,new_euid,CLAIM_ID,cli_dx1_ccs,cli_dx2_ccs,cli_dx3_ccs,cli_dx4_ccs,proc_ccs
499,0578170,141200BTNL00,49.0,NaN,NaN,NaN,243
500,0578170,141200BTNL00,49.0,NaN,NaN,NaN,243
501,0578170,141200CJFL00,49.0,NaN,NaN,NaN,243
505,0578170,141900BMXH00,49.0,NaN,NaN,NaN,243
506,0578170,141980HFPK00,49.0,NaN,NaN,NaN,243


In [15]:
print("unique diabetic patient") 
print(len(all_diabetic['new_euid'].unique())) #unique diabetic patient 

unique diabetic patient
14845


# Making Binary Matrix of diseases 

In [16]:
#Removing NA's in all_diabetic
all_diabetic=all_diabetic.fillna(0)


##changing datatype  
cols=['cli_dx1_ccs','cli_dx2_ccs','cli_dx3_ccs','cli_dx4_ccs']
all_diabetic[cols] = all_diabetic[cols].astype(int)  
 

# Importing CCS Codes

In [18]:
ccs_codes=pd.read_csv('../../Data/ccs_code.csv')  #reading ccs code
ccs_codes['ccs_code']= "CCS"+ ccs_codes.START.map(str) #Adding CCS Codes 
ccs_codes.head() 

,START,LABEL,ccs_code
0,1,Tuberculosis,CCS1
1,10,Immuniz/scrn,CCS10
2,100,Acute MI,CCS100
3,101,Coron athero,CCS101
4,102,Chest pain,CCS102


# Making Empty Binary Matrix

In [19]:
cols=list(ccs_codes['ccs_code'])
cols.insert(0,'new_euid') 
check=pd.DataFrame(columns=cols,index=all_diabetic.index) 
check.head()  

,new_euid,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
##Writing Code to fill up binary Matrix  
for index,row in all_diabetic.iterrows(): 
    if(row['cli_dx1_ccs'] !=0): 
        col="CCS" +str(row['cli_dx1_ccs'])
        check.loc[index][col]=1 
        check.loc[index]['new_euid']=row['new_euid']
    if(row['cli_dx2_ccs'] !=0): 
        col="CCS" +str(row['cli_dx2_ccs'])
        check.loc[index][col]=1 
        check.loc[index]['new_euid']=row['new_euid']
    if(row['cli_dx3_ccs'] !=0): 
        col="CCS" +str(row['cli_dx3_ccs']) 
        check.loc[index][col]=1 
        check.loc[index]['new_euid']=row['new_euid'] 
    if(row['cli_dx4_ccs'] !=0): 
        col="CCS" +str(row['cli_dx4_ccs'])
        check.loc[index][col]=1 
        check.loc[index]['new_euid']=row['new_euid'] 

In [21]:
print(all_diabetic.shape)
print(check.shape)

(96238, 7)
(96238, 284)


In [22]:
check=check.fillna(0)
check_new=check.groupby(check['new_euid']).sum()
print(check_new.shape)  

(14845, 283)


In [23]:
check_new.head()  

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
0000016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000033,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
0000069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
0000104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [24]:
## changing all values greater >1 =1
def f1(x):
    if x>=1:
        return 1
    else:
        return 0

check_new=check_new.applymap(f1)
check_new.head() 

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS107,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
0000016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000033,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
print(check_new.shape)
del check_new['CCS49'] ## Removing CCS49 i,e Diabetes with no complication
del check_new['CCS256'] #Removing Exam/Eval attribute.
check_new=check_new.loc[:,(check_new!=0).any(axis=0)] ## Removing 0 column  
print(check_new.shape) 

(14845, 283)
(14845, 246)


# Making CCS_codes_Summary

In [26]:
### Making CCS Codes with description and population statistics
df=check_new
codes_stats=pd.DataFrame(df.sum())
codes_stats.reset_index(inplace=True) 
codes_stats=codes_stats.rename(columns={0:'Total_Patients'})
codes_stats.head()  

,index,Total_Patients
0,CCS1,1
1,CCS10,459
2,CCS100,45
3,CCS101,735
4,CCS102,322


In [27]:
tmp1=ccs_codes[['ccs_code','LABEL']]
codes_stats=pd.merge(tmp1,codes_stats,left_on='ccs_code',right_on='index')
del codes_stats['index'] 
codes_stats=codes_stats.sort_values(['Total_Patients'],ascending=False)
codes_stats['population_percentage']=(codes_stats['Total_Patients']*1.0/14845)*100.0
codes_stats.head()  

,ccs_code,LABEL,Total_Patients,population_percentage
193,CCS53,Hyperlipidem,8099,54.557090
244,CCS98,HTN,7302,49.188279
190,CCS50,DiabMel w/cm,4946,33.317615
198,CCS58,Ot nutrit dx,3235,21.791849
188,CCS48,Thyroid dsor,1735,11.687437


# Deduplicating Binary Matrix (unique vectors)

In [28]:
cols=check_new.columns.tolist()
binary_IDF=check_new

##Deduplicating binary_IDF
binary_IDF_ddup=pd.DataFrame(binary_IDF.groupby(binary_IDF.columns.tolist(),as_index=False).size())  
print(binary_IDF_ddup.shape)
binary_IDF_ddup.reset_index(inplace=True)  
binary_IDF_ddup=binary_IDF_ddup.rename(columns={0:'Total_Patients'})
binary_IDF_ddup.sort_values(['Total_Patients'],ascending=False,inplace=True)
binary_IDF_ddup.reset_index(inplace=True)
del binary_IDF_ddup['index']
binary_IDF_ddup.head()

(8130, 1)


,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS108,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,Total_Patients
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1225
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,532
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,298
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,255
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,159


# Clustering using IDF & Cosine Similarity 

In [29]:
# #Demonstrating difference between jaccard and cosine after IDF

# from sklearn.metrics.pairwise import cosine_similarity 
# from sklearn.metrics import jaccard_similarity_score

# #Using jaccard with probability of diseases=0.9 ,0.8,0.1 
# print(jaccard_similarity_score([1,1, 0], [0,1,1]))  #AB
# print(jaccard_similarity_score([1,1,0], [1,1,1]))   #AC
# print(jaccard_similarity_score([0,1,1], [1,1,1]))   #BC

# print('\n') 


# #using cosine after IDF = ln(1/prob)
# print(cosine_similarity([0.1, 0.2, 0], [0,0.2,2]))[0][0]   #AB
# print(cosine_similarity([0.1, 0.2, 0], [0.1,0.2,2]))[0][0] #AC
# print(cosine_similarity([0, 0.2, 2], [0.1,0.2,2]))[0][0]   #BC



In [30]:
print(check_new.shape)
check_new.head()

(14845, 246)


,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS108,...,CCS90,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99
new_euid,,,,,,,,,,,,,,,,,,,,,
0000016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000033,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0000104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [31]:
## changing datatype of binary_IDF_ddup
cols=binary_IDF_ddup.columns.tolist()
binary_IDF_ddup[cols]=binary_IDF_ddup[cols].astype(float)


##converting 0,1 to IDF's
for index,row in binary_IDF_ddup.iterrows():
    #print(index)
    for col in cols[:-1]:
         if(row[col]==1):
            fre=codes_stats[codes_stats['ccs_code']==col]['population_percentage']
            fre=(fre*1.0/100).iloc[0]
            #print("frequency is",fre)
            binary_IDF_ddup.loc[index][col]=np.log(1.0/fre)*1.0 
            #print("code is",col,"frequency is" ,fre,"IDF is",np.log(1.0/fre)*1.0,"Valu
binary_IDF_ddup.head()

,CCS1,CCS10,CCS100,CCS101,CCS102,CCS103,CCS104,CCS105,CCS106,CCS108,...,CCS91,CCS92,CCS93,CCS94,CCS95,CCS96,CCS97,CCS98,CCS99,Total_Patients
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1225.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.709515,0.0,532.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,298.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,255.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.709515,0.0,159.0


In [32]:
print(codes_stats[codes_stats['ccs_code']=='CCS98'])
math.log(1.0/0.49188)

    ccs_code LABEL  Total_Patients  population_percentage
244    CCS98   HTN            7302              49.188279


0.7095204946778906

# Performing Clustering Using Cosine Similarity

In [65]:
checkx=binary_IDF_ddup[1:]
X=pdist(checkx.ix[:,0:len(binary_IDF_ddup.columns)-1],metric='cosine')
#Z=linkage(checkx.ix[:,0:len(binary_IDF_ddup.columns)-1],method='complete',metric='cosine') 

Z = linkage(X,method='average') 
c,cd=cophenet(Z,X)
c

# Retrieving the clusters

In [117]:
max_d=0.3
clusters=fcluster(Z,max_d,criterion='distance')
clusters

clusters=pd.DataFrame(clusters)
clusters=clusters.rename(columns = {0:'clusters_id'})  
checkx_new=pd.concat([checkx.reset_index(),clusters], axis=1)  
cols = checkx_new.columns.tolist() 
cols = cols[-2:] + cols[:-2] 
checkx_new=checkx_new[cols] 
checkx_new.set_index('index',inplace=True)  
community=checkx_new.groupby(['clusters_id']).Total_Patients.sum().sort_values(ascending=False)
community[:20]

clusters_id
2672    684.0
2742    575.0
2741    464.0
2743    376.0
2673    298.0
2746    255.0
2736    168.0
2740    150.0
1989    143.0
1643    130.0
2937    110.0
2671    105.0
2730    103.0
3202    103.0
2764     95.0
2138     90.0
1830     88.0
2882     87.0
2499     86.0
2686     84.0
Name: Total_Patients, dtype: float64

In [56]:
def z_score(ccs_code):
    
    p_1=cluster_stats[cluster_stats['ccs_code']==ccs_code]['Total_Patients']*1.0/community[cluster_stats['cluster_id'][0]]
    p_1=p_1.iloc[0]
    p_2=codes_stats[codes_stats['ccs_code']==ccs_code]['Total_Patients']*1.0/14845.0
    p_2=p_2.iloc[0]
    
    n_1 = community[cluster_stats['cluster_id'][0]]*1.0 #cluster Population
    n_2 = 14845.0  #Total Population 
  
    p =(n_1*p_1 + n_2*p_2)*1.0/(n_1 + n_2)
   
    num = (p_1-p_2)*1.0
    deno= math.sqrt(p*(1-p)*( (1.0/n_1) + (1.0/n_2) ))
    
    z=round((num*1.0/deno),2)  
    
    p_values = round((scipy.stats.norm.sf(abs(z))*2),4) #twosided P Tailed test
    
    return(z,p_values)

def clusterstatistic(cluster_id):
    #print(cluster_id)
    cluster=checkx_new[checkx_new['clusters_id']==cluster_id]
   
    cluster=cluster.loc[:,(cluster!=0).any(axis=0)]
    #cluster.head()

    ccs_codes=cluster.columns.tolist()
    ccs_codes=ccs_codes[2:]
    cluster_stats=pd.DataFrame(columns=['cluster_id','CCS_Code','Total_Patients'])
    for code in ccs_codes:
        patients=cluster[cluster[code]!=0].Total_Patients.sum() 
        #print("code is",code,"patients is",patients)
        tmp=pd.DataFrame(index=range(1,2),columns=['cluster_id','CCS_Code','Total_Patients']) 
        tmp.ix[:,0]=cluster_id
        tmp.ix[:,1]=code
        tmp.ix[:,2]=patients 
        cluster_stats=cluster_stats.append(tmp) 

    #cluster_stats.head()

    #cluster_stats=cluster_stats[1:]
    cluster_stats.sort_values(by='Total_Patients',ascending=False,inplace=True)
    #cluster_stats.head()

    cluster_stats['%community']=(cluster_stats['Total_Patients']*1.0/cluster['Total_Patients'].sum())*100.0
    #cluster_stats['%diabetic_population']=(cluster_stats['Total_Patients']*1.0/14845.0)*100.0
    #cluster_stats.head()

    #tmp1.head()

    cluster_stats=pd.merge(tmp1,cluster_stats,left_on='ccs_code',right_on='CCS_Code')
    #cluster_stats.head()

    del cluster_stats['CCS_Code']
    cluster_stats.sort_values(by='Total_Patients',ascending=False,inplace=True)
    return cluster_stats

In [118]:
community[:20]


clusters_id
2672    684.0
2742    575.0
2741    464.0
2743    376.0
2673    298.0
2746    255.0
2736    168.0
2740    150.0
1989    143.0
1643    130.0
2937    110.0
2671    105.0
2730    103.0
3202    103.0
2764     95.0
2138     90.0
1830     88.0
2882     87.0
2499     86.0
2686     84.0
Name: Total_Patients, dtype: float64

# Analyzing Each cluster

In [123]:
cluster_stats=clusterstatistic(1989)
cluster_stats['%diabetic_population']=np.nan
cluster_stats['z_score']=np.nan
cluster_stats['p_value']=np.nan
for index,row in cluster_stats.iterrows():
    z,p=z_score(row['ccs_code'])
    cluster_stats.loc[index,'%diabetic_population']=codes_stats[codes_stats['ccs_code']==row['ccs_code']] .population_percentage.iloc[0] 
    cluster_stats.loc[index,'z_score']=z
    cluster_stats.loc[index,'p_value']=p  
cluster_stats.sort_values(by='z_score',ascending=False,inplace=True)   
cluster_stats

,ccs_code,LABEL,cluster_id,Total_Patients,%community,%diabetic_population,z_score,p_value
2,CCS51,Ot endo dsor,1989.0,143,100,3.475918,56.03,0.0000
5,CCS58,Ot nutrit dx,1989.0,67,46.8531,21.791849,7.20,0.0000
0,CCS48,Thyroid dsor,1989.0,24,16.7832,11.687437,1.88,0.0601
3,CCS52,Nutrit defic,1989.0,16,11.1888,9.195015,0.82,0.4122
4,CCS53,Hyperlipidem,1989.0,58,40.5594,54.557090,-3.34,0.0008
6,CCS86,Cataract,1989.0,1,0.699301,9.834961,-3.67,0.0002
7,CCS98,HTN,1989.0,39,27.2727,49.188279,-5.22,0.0000
1,CCS50,DiabMel w/cm,1989.0,12,8.39161,33.317615,-6.30,0.0000
